## Frame

In [178]:
using Gridap
using Plots
using GridapMakie
using CairoMakie 
using GLMakie
using Gridap.Geometry
using GridapGmsh

In [179]:
# Define known parameters/functions

# Nacelle
F_Nacelle = 5000*9.81 # N

# Pile 
m_Pile = (1000*9.81) # N/m
EI_Pile = 1e9 # Nm2
EA_Pile = 1e8 # N

# Jacket 
m_Jacket = (100*9.81) # N/m 
EI_Jacket = 1e7 # Nm2 
EA_Jacket = 1e6 # N 

model = GmshDiscreteModel("msh_files/OFFSHORE_NASAL_JACKET.msh")

Info    : Reading 'msh_files/OFFSHORE_NASAL_JACKET.msh'...
Info    : 48 entities
Info    : 446 nodes
Info    : 456 elements
Info    : Done reading 'msh_files/OFFSHORE_NASAL_JACKET.msh'


UnstructuredDiscreteModel()

In [180]:
# Set up the figure and axis 
labels = get_face_labeling(model)
initial_tags = get_tag_entities(labels)[end]
tag_from_names = get_tag_from_name(labels)

Dict{String, Int64} with 5 entries:
  "Jacket Members" => 4
  "Pile Members"   => 5
  "Supports"       => 1
  "Bottom Pile"    => 3
  "Nacelle"        => 2

In [181]:
# Set up the figure and axis 
fig = Figure(resolution=(800, 600))
ax = Axis(fig[1, 1], title="Domain with Boundary Tags", xlabel="x", ylabel="y")

for intgr in length(tag_from_names)
    CairoMakie.wireframe!(ax, Triangulation(model))
end

CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Supports"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Jacket Members"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Pile Members"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Bottom Pile"))

# fig

Scatter{Tuple{Vector{GeometryBasics.Point{2, Float64}}}}

In [182]:
# add_tag_from_tags!(labels, "Supports", [1]) 
# add_tag_from_tags!(labels, "Bottom Pile", [3])
# add_tag_from_tags!(labels, "Nacelle", [2])

In [184]:
Ω = Triangulation(model)
Ω_J = Triangulation(model, tags="Jacket Members")
Ω_P = Triangulation(model, tags="Pile Members")
Γ_S = Boundary(Ω,tags="Supports")
Γ_BP = Boundary(Ω,tags="Bottom Pile")
Γ_NC = Boundary(Ω,tags="Nacelle")

Λ = Skeleton(Ω)

CompositeTriangulation()

In [185]:
order_u = 1
order_v = 2

reffe_u = ReferenceFE(lagrangian, Float64, order_u) 
reffe_v = ReferenceFE(lagrangian, Float64, order_v) 

Y_u = TestFESpace(Ω, reffe_u, dirichlet_tags="Supports") # The TestFESpace for the rod equation
X_u = TrialFESpace(Y_u, 0.0) # The TrialFESpace for the rod equation

Y_v = TestFESpace(Ω, reffe_v, dirichlet_tags="Bottom Pile") # The TestFESpace for the beam equation
X_v = TrialFESpace(Y_v, 0.0) # The TrialFESpace for the rod equation

X = MultiFieldFESpace([X_u, X_v])
Y = MultiFieldFESpace([Y_u, Y_v])

MultiFieldFESpace()

In [186]:
degree_u = 2*order_u
degree_v = 2*order_v 

dΩ_r = Measure(Ω,degree_u) # Numerical integration within the domain Ω using second-order Gaussian quadrature rule
dΓr_N = Measure(Γ_NC,degree_u) # Numerical integration along the Neumann boundary Γ_D using second-order Gaussian quadrature rule

dΩ_b = Measure(Ω,degree_v) # Numerical integration within the domain Ω using second-order Gaussian quadrature rule
dΓb_N = Measure(Γ_NC,degree_v) # Numerical integration along the Neumann boundary Γ_D using second-order Gaussian quadrature rule

nΓₗ = get_normal_vector(Γ_NC)

dΛ = Measure(Λ,2)
nΛ = get_normal_vector(Λ) # Normal vector to the Neumann boundary Γ_N

SkeletonPair()

In [ ]:
# Weak form
# a((u,v), (w,h)) = ∫(EA_Pile * ∇(u) ⋅ ∇(w))dΩ_r + ∫( EI_Pile*Δ(v)*Δ(h))dΩ_b

a((u,v), (w,h)) = ∫( EI_Jacket*(Δ(u)*Δ(w)) )dΩ_b +
  ∫( EI_Jacket*(jump(∇(u)⋅nΛ)*jump(∇(w)⋅nΛ)) -
    mean(EI_Jacket*Δ(u))*jump(∇(w)⋅nΛ) -
    mean(EI_Jacket*Δ(w))*jump(∇(u)⋅nΛ) )dΛ + ∫(EA_Pile * ∇(u) ⋅ ∇(w))dΩ_r

l((w,h)) = ∫(m_Jacket * w)dΩ_b + ∫(m_Pile * w)dΩ_r + ∫(F_Nacelle * h)dΓr_N

op = AffineFEOperator(a,l,X,Y)

AffineFEOperator()

In [177]:
# Solve the problem
# op = AffineFEOperator(a, l, U, V)

uh, vh = solve(op)
writevtk(Ω,"2d_frame",cellfields=["u"=>uh, "v"=>vh])

LinearAlgebra.SingularException: LinearAlgebra.SingularException(0)